<a href="https://colab.research.google.com/github/nedvb/Recurrent-Neural-Network-Text-Generator/blob/main/Recurrent%20Neural%20Network%20Text%20Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import string

In [ ]:
!git clone https://github.com/nedvb/ML-Project-4.git

Cloning into 'ML-Project-4'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 21.71 KiB | 1.55 MiB/s, done.


In [ ]:
%cd ML-Project-4/

/content/ML-Project-4


In [ ]:
DATA_FILE = 'alice.txt'
SEQ_LENGTH = 100
HIDDEN_DIM = 700
LAYER_NUM = 3
BATCH_SIZE = 12
GENERATE_LENGTH = 20

data = open(DATA_FILE, 'r', encoding='latin-1').read()

valid_characters = string.ascii_letters + ".,! -'" + string.digits
char_to_int = {ch: i for i, ch in enumerate(valid_characters)}
int_to_char = {i: ch for i, ch in enumerate(valid_characters)}

# Clean and process the text
data_cleaned = ''.join([ch if ch in valid_characters else ' ' for ch in data])
data_cleaned = data_cleaned.replace('  ', ' ')

# Create input-output pairs
sequences = [data_cleaned[i:i + SEQ_LENGTH + 1] for i in range(0, len(data_cleaned) - SEQ_LENGTH, SEQ_LENGTH)]


class CharLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(CharLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(out.view(-1, self.hidden_size))
        return out, h

    def init_hidden(self, batch_size):
        # Initialize hidden state and cell state with zeros
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return (hidden, cell)


# Initialize the model
model = CharLSTM(len(valid_characters), HIDDEN_DIM, len(valid_characters), LAYER_NUM)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def generate_text(model, start_char='A', length=GENERATE_LENGTH):
    model.eval()
    chars = [ch for ch in start_char]

    # Initialize hidden state with batch size of 1
    h = model.init_hidden(1)

    x = torch.zeros(1, 1, len(valid_characters))
    x[0, 0, char_to_int[start_char]] = 1

    for _ in range(length):
        out, h = model(x, h)

        # Sample the output distribution to get the predicted character
        out_dist = out.view(-1).div(0.8).exp()
        top_i = torch.multinomial(out_dist, 1)[0]
        predicted_char = int_to_char[top_i.item()]

        chars.append(predicted_char)

        # Update the input for the next character prediction
        x = torch.zeros(1, 1, len(valid_characters))
        x[0, 0, top_i] = 1

        # Ensure the hidden state maintains the correct batch size
        h = (h[0][:, :1, :], h[1][:, :1, :])

    return ''.join(chars)


# Training loop
for epoch in range(13):
    model.train()
    for sequence in sequences:
        inputs = torch.zeros(1, SEQ_LENGTH, len(valid_characters))
        targets = torch.zeros(SEQ_LENGTH, dtype=torch.long)

        for i in range(SEQ_LENGTH):
            inputs[0, i, char_to_int[sequence[i]]] = 1
            targets[i] = char_to_int[sequence[i + 1]]

        optimizer.zero_grad()
        h = model.init_hidden(1)  # Initialize hidden state for batch size of 1
        outputs, h = model(inputs, h)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/13], Loss: {loss.item():.4f}')

print(generate_text(model, start_char='A'))



Epoch [1/13], Loss: 2.0126
Epoch [2/13], Loss: 1.7146
Epoch [3/13], Loss: 1.5286
Epoch [4/13], Loss: 1.3076
Epoch [5/13], Loss: 1.0793
Epoch [6/13], Loss: 0.9466
Epoch [7/13], Loss: 0.8881
Epoch [8/13], Loss: 0.6729
Epoch [9/13], Loss: 0.5512
Epoch [10/13], Loss: 0.5147
Epoch [11/13], Loss: 0.3811
Epoch [12/13], Loss: 0.2594
Epoch [13/13], Loss: 0.2433
Alice heard the King.
